# Homework 3
### Group number 26

#### Packages used
In the following cell all the packages needed to run this project are loaded.  
The $functions.py$ file contains hand made functions that for the sake of tidyness we chose not to present in the notebook.

In [1]:
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
from bs4 import BeautifulSoup
from functools import reduce
import plotly.express as px
import urllib.request
import pandas as pd
import numpy as np
import functions
import requests
import string
import heapq
import json
import nltk
import time
import lxml
import bs4
import sys
import os

## Data Collection
In this section the data of interest is retrieved from the website $Atlas Obscura$.

#### 1.1 Get the list of places
At first we listed all the URLs of the places in the $Pages.txt$ file. The url is split in two pieces, because the only characters that change are the ones referring to the number of the page.

In [2]:
URL="https://www.atlasobscura.com/places?page=1&sort=likes_count"
url1="https://www.atlasobscura.com/places?page=" 
url2="&sort=likes_count"

functions.pull_out(url1,url2,URL)

#### 1.2 Crawl places
The following cell visits each Url contained in $Pages.txt$ and downloads its html code, these files are progressively stored in the Folders as instructed.  

In [3]:
f = open("Pages.txt", "r")
k=0

for line in f:    
    path = os.getcwd()
    directory = "\\Data\\Page"+str(int(k//18+1))+"\\"
    os.makedirs(path+directory,exist_ok=True)
    response = urllib.request.urlopen(line)
    html = response.read()
    with open(path+directory+line[36:-1].replace("-"," ")+".html","wb")as x:
        x.write(html)        
    k+=1    

#### 1.3 Parse downloaded pages
Once the process of $point 1.2$ is completed, all the downloaded html pages are analized, the required informations are stored in a $csv$ file called $AO$_$Data$.  

In [2]:
functions.DFBuilder()

## Search Engine

### 2.1 Conjunctive query
The text preprocessing steps that we decided to apply to our dataset are the following:
1. Removing punctuation: alongside the punctuation set of the string package we added some custom ones we found in the data
2. Removing stopwords
3. Tokenizing and lowering
4. Three steps lemmatization: once with default settings, once for verbs and once for adjectives, the motivation behind this choice is that the user might want to visit a place that offers a particular activity, so it is crucial to find the associated verbs  

#### 2.1.1 
In the following cell the dataset we created in $point 1.3$ is loaded in memory, also two lists are defined that will be important for leter steps:   
$wordlist$ contains the processed list of tokens of the place description of each row of the dataset, $word$ is a list of all the unique words found across all those documents.  

In [2]:
aodf = pd.read_csv("AO_Data.csv")

nltk.download("stopwords")
nltk.download('wordnet')

wordlist = aodf.apply(lambda row: functions.TextPre(row["placeDesc"]), axis=1)
words = set(reduce(lambda x, y: x+y, wordlist))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Leythred\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Leythred\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Now we proceed to computing the $word$ $to$ $id$ and the $inverted$ $index$ dictionaries.  
In the process, a new column is added to the loaded dataframe called $listwords$ that contains the output of the pre-processing procedure, this column is used as an auxiliary container to compute the $inverted$ $index$ dictionary.  
Then the dictionaries are saved for convenience.

In [3]:
# Building of the word to id dictionary
k=1
dictio = {}
for i in words:
    dictio[i] = k
    k +=1
    
aodf['listwords'] = aodf.placeDesc.apply(lambda row: functions.TextPre(row))

# Building of the inverted index dictionary
invertedidx = {}
for k, v in dictio.items():
    invertedidx[v] = list(aodf[aodf.listwords.apply(lambda row: k in row)].index)


In [4]:
# Saving of the dictionaries to txt files
with open('nvtdidx.txt', 'w') as f:
     f.write(json.dumps(invertedidx))

with open('idwordsdict.txt', 'w') as f:
     f.write(json.dumps(dictio))

#### 2.1.2
In the following cell is presented the code for the first search engine:  The input received from the user is pre-processed with se same function used before, then the token of the query are converted in ids using the $word$ $to$ $id$ dictionary when found inside it. This search engine gives an output only if all af the inputed words are found in the $word$ $to$ $id$ dictionary and consequently in the collection of documents.

In [2]:
# Loading of the two dictionaries previously computed (provided in the repository)
with open('nvtdidx.txt') as f:
    data = f.read()
invertedidx = json.loads(data)

with open('idwordsdict.txt') as f:
    data = f.read()
dictio = json.loads(data)
# !!Run this cell before proceeding further!!

In [105]:
# First Search Engine
query = functions.TextPre(" ".join(input().split()))
queryids = []
reslist = []
for i in query:
    if i not in dictio.keys():
        queryids.append(None)
    else:
        queryids.append(dictio[i])
if None not in queryids:           #in this cycle the sets made from the list in the invertedidx values are added to a list
    for j in range(len(queryids)): #then it is taken the intersection between all of them    
        reslist.append(set(invertedidx[str(queryids[j])])) 
        result = set.intersection(*reslist)
    if len(result) == 0: print("No Results Found")
    else: display(aodf.loc[result,["placeName","placeDesc","placeUrl"]])    
else: print("No Results Found")

beach sea mexico


,placeName,placeDesc,placeUrl
939,Clam Pass Beach,"On the coast of Naples, Florida, where a for...",https://www.atlasobscura.com/places/clam-pass-...


### 2.2 Conjunctive query & Ranking score
In this section the $tf$$idf$ score is used to provide more depth to the results of the query.

#### 2.2.1
In the following cell the $tf$$idf$ scores are computed.  
The $wordlist$ variable defined before (and presented here again for convenience) is used from now on to help in the computations, its main purpose is to keep track of the document $id$ we are working on. In the specific case of the computation of the $tf$$idf$ scores it also carries the information of the length of the document, used to compute the $tf$ coefficient.  
In the calculation of the $idf$ value we leveraged the structure of the $inverted$ $index$ dictionary in order to easily compute it in line 9 of the following cell.

$$
tfidf_{i, j}=t f_{i, j} \times \log \left(\frac{N}{d f_i}\right)
$$
$tf_{i j}=$ number of occurrences of $i$ in $j$,
$d f_i=$ number of documents containing $i$,
$N=$ total number of documents.

In [3]:
# Building of the tfidf coefficients of each word
aodf = pd.read_csv("AO_Data.csv")
wordlist = aodf.apply(lambda row: functions.TextPre(row["placeDesc"]), axis=1)
words = set(reduce(lambda x, y: x+y, wordlist))
tfidf = {}
for word in words:
    for i in invertedidx[str(dictio[word])]:
        count = Counter(wordlist[i])   
        freq = count[word]
        tf = freq/len(wordlist[i])
        df = (len(wordlist)/len(invertedidx[str(dictio[word])]))
        idf = np.log(df)
        tfidf[str(dictio[word]), i] = tf*idf       

Now we build and save the new requested $inverted$ $index$ dictionary with the addition of the $tf$$idf$ scores, although it is not needed in the following steps as the algorithm I created only uses the $tf$$idf$ dictionary.

In [5]:
# Addition of the tfidf to the inverted index dictionary
indict_fidf = {}
for i in invertedidx.keys():
    row = []
    for j in invertedidx[i]:
        t = [j, tfidf[i,j]]
        row.append(t)
    indict_fidf[i] = row
    
with open('nvtfidfidx.txt', 'w') as f:
     f.write(json.dumps(indict_fidf))

#### 2.2.2
In this section the cosine similarity between the inputed query and the documents will be computed.  
  
In order to vectorize the documents it has been created a function called $NVD$ (see functions.py for details), this function takes as arguments: a list, an integer, the $tf$$idf$ dictionary, and the $word$ $to$ $id$ dictionary.  
The $NVD$ is made to take the elements of $wordlist$ iteratively and outputting for each the vectorized document normalized to 1.  
  
These vectors are then stored in a dictionary that has the document id as key.

In [5]:
# Building the dictionary of the vectorized documents
vectors = {}
for i in range(len(wordlist)):
    vectors[i] = functions.NVD(wordlist[i], i, tfidf, dictio)

The following cell contains the search engine.  
  
It uses a function called $NVQ$ that vectorizes and normalizes the inputed query, then the cosine between the query and all the documents in $vectors$ is computed, all the non zero results are put inside a heap data structure as tuples containing the computed cosine and the relative document id.  
  
Finally, the top 15 results are shown, the similarity score is calculated normalizing the cosine value relative to each result to the cosine value of the highest rated one.

In [60]:
# Second Search Engine

# Preprocessing and vectorizing the query
query = functions.TextPre(" ".join(input().split()))
querytxt = []
for i in query:
    if i in dictio.keys():
        querytxt.append(i)
    else:
        continue
queryvector = functions.NVQ(querytxt, dictio)

# Computing the cosine similarity between the vectorized query and documents
heap = []
for i in range(len(vectors)):
    cos = np.dot(queryvector, vectors[i])
    if cos != 0:
        heapq.heappush(heap,(cos, i))

# Output of the top 15 results of the query
k = 15
topk = heapq.nlargest(k, heap)
res = []
sim = []
for i in range(len(topk)):
    res.append(topk[i][1])
    sim.append(topk[i][0])
topkresults = pd.DataFrame(aodf.loc[res,["placeName","placeDesc","placeUrl"]])

if len(sim) != 0:
    topkresults["similarity"] = sim/sim[0]
    display(topkresults)
else: print("No Results Found")

beach sea


,placeName,placeDesc,placeUrl,similarity
4774,Hidden Beach,Update as of July 2021: This location is now...,https://www.atlasobscura.com/places/hidden-beach,1.000000
4021,Kauai Glass Beach,A tiny little beach nestled between an indus...,https://www.atlasobscura.com/places/kauai-glas...,0.812350
4010,Boyeeghter Bay,This gorgeous hidden beach is a rugged slive...,https://www.atlasobscura.com/places/murder-hol...,0.765846
4088,International Sea Glass Museum,What appears to be a kitschy sea glass gift ...,https://www.atlasobscura.com/places/internatio...,0.617738
4225,Sometimes Train Tracks of Sunset Beach,"In early November of 2014, a colossal storm ...",https://www.atlasobscura.com/places/sometimes-...,0.616754
6975,Red Beach at Mjelle,The military associated town of Bodø is not ...,https://www.atlasobscura.com/places/red-beach-...,0.601402
1469,Punalu`u Black Sand Beach,Thanks to lava streams beneath the water jus...,https://www.atlasobscura.com/places/punaluu-bl...,0.597932
3367,Crescent Beach,"Hidden behind two regal viewing points, Cres...",https://www.atlasobscura.com/places/crescent-b...,0.585843
1845,Iona's Beach,Iona’s Beach is a hidden gem. Instead of sil...,https://www.atlasobscura.com/places/ionas-beach,0.561390
5960,Bombay Beach,"In 2010, the United States Census Bureau mea...",https://www.atlasobscura.com/places/bombay-beach,0.536918


## Define a new score

The extra information that the user can input will be the location to which he/she is interested to go to.  
In order to include this new information in the scoring system, I created a new parameter similar to the $tf$$idf$ score, I only considered the inverse document frequency coefficient score of each word in the address field of the dataset, because the term frequency in this case is basically mute.  
The skeleton of the process is the same as the one already explained to get the tfidf scores.

#### 3.1 Preprocessing
The placeAddress field of the dataset is processed by removing puncuation, stopwords and the words that contain numbers, no lemmatization or stemming procedures were employed as t5hey are irrelevant in this case. Words containing numbers were removed in order to avoid accidental matches with only that portion of the address, which would have been nonsensical.  
The function used in this step is $AddPre$ in the $functions.py$ file.

#### 3.2 Auxiliary lists and dictionaries
In the following cells are created the lists needed by the search engine algorithm and the new dictionaries relative to the address field,the $Addressword$ $to$ $id$ and the $Address$ $inverted$ $index$ dictionaries.  
In the last cell it is computed the score dictionary called $add$$idf$.

In [6]:
# Auxiliary lists
aodf['listadd'] = aodf.placeAddress.apply(lambda row: functions.AddPre(row))
addlist = aodf.apply(lambda row: functions.AddPre(row["placeAddress"]), axis=1)
adds = set(reduce(lambda x, y: x+y, addlist))

In [7]:
# Auxiliary dictionaries
k=1
dictio_add = {}
for i in adds:
    dictio_add[i] = k
    k +=1

invertedidx_add = {}
for k, v in dictio_add.items():
    invertedidx_add[v] = list(aodf[aodf.listadd.apply(lambda row: k in row)].index)

In [8]:
# New score dictionary
addidf = {}
for add in adds:
    for i in invertedidx_add[dictio_add[add]]:
        df = (len(addlist)/len(invertedidx_add[dictio_add[add]]))
        idf = np.log(df)/10
        addidf[str(dictio_add[add]), i] = idf  

#### 3.3 Vectorization
In the following cell we build a new set of vectorized documents, now in the space of the tokens found in the address field.

In [9]:
vectors_add = {}
for i in range(len(addlist)):
    vectors_add[i] = functions.NVD_add(addlist[i], i, addidf, dictio_add)

#### 3.4 Search Engine
The search engine receives two inputs from the user, one is the string of key words and the other is a location of interest that can be a city or a state. These inputs are vectorized respectevely in the space of words of $placeDesc$ and $placeAddress$. Then the cosine similarities are computed, using the vectorized documents of each space.  
  
The final scoring system is the following:  
The priority is given to the the input location, the reason being that if the user is planning a trip to a specific country it's not logical to recommend places elsewhere, even if said places coincide with the interests of the user. In addition, if the user wants recommendations without factoring the locations in, he/she may just use the second search engine.  
The similarity score is obtained by summing the two cosine similarities computed in the vectorial spaces of descriptions and addresses, a factor $a$ is used to give different weights to them, the specific value chosen is the result of its hand tuning in some edge cases found.   

In [61]:
# Third Search Engine

query = functions.TextPre(" ".join(input().split()))
querytxt = []
for i in query:
    if i in dictio.keys():
        querytxt.append(i)
    else:
        continue
# Vectorization of the first part of the query in the space of placeDesc
queryvector = functions.NVQ(querytxt, dictio)

print("Type the location")
query_add = functions.AddPre(" ".join(input().split()))
querytxt_add = []
for i in query_add:
    if i in dictio_add.keys():
        querytxt_add.append(i)
    else:
        continue
# Vectorization of the second part of the query in the space of placeAddress
queryvector_add = functions.NVQ(querytxt_add, dictio_add)

heap_add = []
# Computation of the score
a = 2.5
for i in range(len(vectors)):
    cos = np.dot(queryvector, vectors[i])
    cos_add = np.dot(queryvector_add, vectors_add[i])
    if cos_add != 0:
        heapq.heappush(heap_add,(a*cos + cos_add, i))
    if cos != 0:
        heapq.heappush(heap_add,(cos, i))

# Output the top 15 results
k = 15
topk_add = heapq.nlargest(k, heap_add)
res = []
sim = []
for i in range(len(topk_add)):
    res.append(topk_add[i][1])
    sim.append(topk_add[i][0])
topkresults = pd.DataFrame(aodf.loc[res,["placeName","placeDesc","placeUrl"]])

if len(sim) != 0:
    topkresults["similarity"] = sim/sim[0]
    display(topkresults)

beach sea
Type the location
mexico


,placeName,placeDesc,placeUrl,similarity
364,Hidden Beach,A gaping hole in the surface of the lush gre...,https://www.atlasobscura.com/places/hidden-bea...,1.000000
1259,Trinity Atomic Bomb Site,"On July 16, 1945, deep in the deserts of New...",https://www.atlasobscura.com/places/trinity-at...,0.669369
106,White Sands National Park,The wind blown sands and rolling dunes of Wh...,https://www.atlasobscura.com/places/white-sand...,0.635615
4774,Hidden Beach,Update as of July 2021: This location is now...,https://www.atlasobscura.com/places/hidden-beach,0.615311
3611,Little Beaver Town,The infamous Red Ryder BB gun from A Christm...,https://www.atlasobscura.com/places/little-bea...,0.605636
4710,Santa Fe Shatter Cones,"About 1.5 billion years ago, the site of wha...",https://www.atlasobscura.com/places/santa-fe-s...,0.604319
2036,Crow Canyon Petroglyphs,This rocky swath of land in New Mexico is ho...,https://www.atlasobscura.com/places/crow-canyo...,0.586270
2552,Prehistoric Trackways National Monument,"In the much loved city of Las Cruces, outdoo...",https://www.atlasobscura.com/places/prehistori...,0.566279
4801,Mine Shaft Tavern,"In a town of about 204 residents, you might ...",https://www.atlasobscura.com/places/the-mine-s...,0.551717
6155,Alamogordo Landfill,While there is some speculation as to whethe...,https://www.atlasobscura.com/places/alamogordo...,0.547319


## Visualizing the most relevant places

A Mapbox plot is used to visualize the locations of the top results of the query on a map of the planet.

In [101]:
# Some small regularizations to the required fields of the dataset
topkresults = pd.DataFrame(aodf.loc[res,["placeName","placeAlt","placeLong", "placeUrl", "numPeopleVisited"]])
topkresults["numPeopleVisited"]=topkresults.numPeopleVisited.apply(lambda row: int(row))
topkresults["placeLat"]=topkresults["placeAlt"].str.replace(",", "")
topkresults["placeLat"]=topkresults.placeLat.apply(lambda row: float(row))
topkresults["placeLong"]=topkresults.placeLong.apply(lambda row: float(row))

centerlat = [i for i in topkresults["placeLat"]]
centerlon = [i for i in topkresults["placeLong"]]

In [102]:
# Map
px.set_mapbox_access_token("pk.eyJ1IjoibGV5dGhyZWQiLCJhIjoiY2xhbnhlbWdrMHRqeDN5b2J2MnJjdHZrMiJ9.tPPsraPrqqbR6HXwrbN74g")
fig = px.scatter_mapbox(topkresults, lat="placeLat", lon="placeLong", center={"lat": centerlat[0],"lon": centerlon[0] },
                        hover_name="placeName", size="numPeopleVisited", mapbox_style="outdoors")
fig.show()

## Theoretical question

#### 7.1 Try solving the problem mentioned above using three different sorting algorithms
The three algorithms we will try to implement are: selection sort, bubble sort and merge sort.

In [1]:
#define a function for the mean with 2 decimals 
def _mean_(values,n):
    return round(sum(values)/n,2)

#function to remove double list
def unlist(studenti):
    for i in studenti:
        print(*i) 

In [3]:
#SELECTION SORT
def sel_sort(values,N):
    # Traverse through all array elements
    for i in range(N):
        # Find the max element in 
        # unsorted array
        max_i = i
        for  j in range(i+1,N):
            if( values[max_i][2] < values[j][2]):
                max_i = j
            elif (values[max_i][2] == values[j][2]):
                if((values[max_i][0]+values[max_i][1]).lower() > (values[j][0] +values[j][1]).lower()):
                    max_i = j
        
        # Swap the found max element with
        # the first element 
        values[i],values[max_i] = values[max_i],values[i]
    return values

In [48]:

with open('ApplicantsInfo.txt','r',encoding='utf-8') as f:
    line = f.readline().split()
    N = int(line[0])
    M = int(line[1])
    rows = []
    for _ in (range(N)):
        row = f.readline().split()
        media = _mean_([int(x) for x in row[2:M+2]],M)
        rows.append([row[0],row[1],media])        

In [49]:
#display our list of students
unlist(rows[:3])

Richard Scott 23.83
Paula Ahmadi 24.02
Arthur Richardson 23.77


In [7]:
start_selection=time.time()
sel_sort(rows,N)
end_selection=time.time()

In [1]:
#MERGE SORT

def mer_sort(rows):         
    if(len(rows)>1):
        m=len(rows)//2 #select the pivot as the mid of the array 
        # Dividing the array elements
        a1=rows[:m]
        a2=rows[m:]
        #recursive function (in this case the recursion cost O(n/2) for each recursion of mer_sort function)
        #to sort the first and second  half
        mer_sort(a1)
        mer_sort(a2)
        #initialize index
        i=0                                                  
        j=0
        k=0
        while i < len(a1) and j < len(a2):
            if a1[i][2] > a2[j][2]:
                rows[k] = a1[i]
                i += 1
            elif (a1[i][2] == a2[j][2]):
                if((a1[i][0]+a1[i][1]).lower() > (a2[j][0] +a2[j][1]).lower()): #condition for alphabetical order, remeber to trasform in lower characters, not capital
                    rows[k]=a2[j]
                    j+=1
                    
                else:
                     rows[k] = a1[i]
                     i += 1
                    
            else:
                rows[k] = a2[j]
                j += 1
            k += 1
            
        # Checking if any element was left    
        while (i < len(a1)):
            rows[k]=a1[i]
            k+=1
            i+=1

        while(j < len(a2)):
            rows[k]=a2[j]
            k+=1
            j+=1

In [51]:
start_merge=time.time()
mer_sort(rows)
end_merge=time.time()

print(unlist(rows[:10]))    # running time O(n*logn)

Emily Crispin 24.49
Patricia Witten 24.49
Bruce Johnson 24.45
Doreen Richmond 24.45
David Niederberger 24.44
Keisha Keene 24.44
Steven Boston 24.44
John Johnson 24.42
Marvin Ramirez 24.42
Melody Sanchez 24.42
None


In [ ]:
#Let's create txt file with sorted students by score 
with open('RankingList.txt','w') as f:
    for i in rows: #rows is sorted after merge sort
        f.write(i[0]+' '+ i[1] +' '+str(i[2]) +'\n')

In [10]:
#BUBBLE SORT... "is the wrong way to go" -cit Barack Obama
def bubbleSort(arr):
    n = len(arr)
     # Traverse through all array elements
    for i in range(len(arr)):
        # Last i elements are already in place
        for j in range(0, n-i-1):
            # traverse the array from 0 to n-i-1
            # Swap if the element found is greater
            # than the next element
            if arr[j][2] < arr[j+1][2]:
                (arr[j], arr[j+1]) = (arr[j+1], arr[j])
            #condition for alphabetical order
            elif (arr[j][2] == arr[j+1][2]):
                if ((arr[j+1][0]+arr[j+1][1]).lower() < (arr[j][0]+arr[j][1]).lower()):
                    (rows[j],rows[j+1]) = (rows[j+1],rows[j])
             

    print("Sorted array is:")
    for i in range(len(arr)):
        print(*arr[i], end="\n")


In [11]:
start_bubble=time.time()
bubbleSort(rows) #O(n^2)
end_bubble=time.time()

Sorted array is:
Emily Crispin 24.49
Patricia Witten 24.49
Bruce Johnson 24.45
Doreen Richmond 24.45
David Niederberger 24.44
Keisha Keene 24.44
Steven Boston 24.44
John Johnson 24.42
Marvin Ramirez 24.42
Melody Sanchez 24.42
Edith Lehtonen 24.41
Ida Mccabe 24.41
Joshua Reece 24.41
Luisa Young 24.41
Manuel Sullinger 24.41
Betty Kubiak 24.4
Cara Baird 24.4
Mattie Salinas 24.4
Particia Mirabal 24.4
Violet Paulino 24.4
Amy Walker 24.39
Harry Lupu 24.39
Jeffrey Johnson 24.39
Josephine Young 24.39
Kathleen Whaley 24.39
Kelsey Mcneill 24.39
Latoya Stemp 24.39
Marie Wall 24.39
Richard Parker 24.39
Amanda Williams 24.38
Daryl Singer 24.38
David Kerns 24.38
Desiree Paul 24.38
Jackson Klopfer 24.38
Jeramy Galicia 24.38
Juan Gonzalez 24.38
Kyle Carlson 24.38
Mary Nguyen 24.38
Norma Smith 24.38
Olive Sacco 24.38
Rose Mcpeak 24.38
Ruth Heimsness 24.38
David Yang 24.37
Edward Corella 24.37
Irvin Johnson 24.37
James Pal 24.37
Khalilah Hensley 24.37
Nancy Leonard 24.37
Rhonda Bennett 24.37
Robert Rect

Brooke Athearn 24.01
Bruce Ingle 24.01
Bryan Chaney 24.01
Calandra Mcclellan 24.01
Calvin Whitehead 24.01
Cameron Matthews 24.01
Camilla Davison 24.01
Carey Jenkins 24.01
Carina Le 24.01
Carl Anderson 24.01
Carl Dewing 24.01
Carlos Murray 24.01
Carlton Fleenor 24.01
Carl True 24.01
Carly Farah 24.01
Carmen Long 24.01
Carmen Miller 24.01
Carmon Vaughn 24.01
Carol Bishop 24.01
Carole Cobb 24.01
Carol Goldberg 24.01
Caroline Skinner 24.01
Carol Kluz 24.01
Carolyn Ramirez 24.01
Carolyn Werth 24.01
Carrie Fritsch 24.01
Carrie Williams 24.01
Casey Harris 24.01
Casey Hobbs 24.01
Catherine Baldwin 24.01
Catherine Martinez 24.01
Catherine Sheffield 24.01
Cecelia Bronson 24.01
Chad Ball 24.01
Chad Harper 24.01
Chandra Taylor 24.01
Charlene Ramirez 24.01
Charles Backhaus 24.01
Charles Browning 24.01
Charles Clark 24.01
Charles Gerke 24.01
Charles Jamison 24.01
Charles Morgan 24.01
Charles Powell 24.01
Charles Sickafoose 24.01
Charles Sidney 24.01
Charles Steelman 24.01
Charles West 24.01
Charles 

Ernest Mcfadden 23.63
Gerald Worthey 23.63
Jimmy Strickland 23.63
Jordon Mclarney 23.63
Joyce Chang 23.63
Lamar Cuevas 23.63
Lori Slaughter 23.63
Mary Johnson 23.63
Richard Peters 23.63
Ruby Sullivan 23.63
Stacey Mays 23.63
Willie Goyette 23.63
Adam Desilva 23.62
Ann Wright 23.62
Anthony Rosenthal 23.62
Bernice Williams 23.62
Carrie Brester 23.62
Clara Jackson 23.62
Gayla Ward 23.62
Heather Brown 23.62
Irving Maccarone 23.62
Lisa Lollar 23.62
Lou Martins 23.62
Solomon Bombard 23.62
Valeria Lewin 23.62
Zelda Gray 23.62
Ginger Rice 23.61
Lorna Young 23.61
Ty Settle 23.61
Andrew Ovellette 23.6
Gregory Johnson 23.6
Pamela Evans 23.6
Philip Barker 23.6
Donald Benavidez 23.59
Louis Miller 23.59
Nathan Berenguer 23.59
Catherine Fulop 23.58
Christy Mendoza 23.58
George Gordon 23.58
Helen Branch 23.58
Max Beck 23.58
Ray Bray 23.57
Eric Schmidt 23.56
Kenneth Marquez 23.56
Ruth Watts 23.56
Anna Martin 23.55
Justin Williams 23.55
Bruce Murphy 23.54
Mary Thompson 23.51


#### 7.2 What is the time complexity of each algorithm you have used?

-MERGE SORT: **O(n*logn)** 

-SELECTION SORT: **O(n^2)**

-BUBBLE SORT: **O(n^2)**

#### 7.3 Evaluate the time taken for each of your implementations to answer the query stored in the ApplicantsInfo.txt file and visualize them.

In [12]:
#display the running time of our algorithms
print('Selection sort need', round(end_selection-start_selection,0) ,'seconds')
print('Merge sort need', round(end_merge-start_merge,2) ,'seconds')
print('Bubble sort need', round(end_bubble-start_bubble,0) ,'seconds')

Selection sort need 514.0 seconds
Merge sort need 0.53 seconds
Bubble sort need 1742.0 seconds


#### 7.4 What is the most optimal algorithm, in your opinion, and why?

The optimal algorithm among those implemented above is ***merge sort***, that is a *Superlinear algorithm – O(nlogn)*. Selection and bubble are both *Polynomial algorithms – O(n^c)*.
Merge sort is the most efficient one as it's running time, also in the worst case,  is O(nlogn).

If we simply analyse the for loops in each algorithm we can see that in both bubble and selection sort there are two nested for loops *(O (n^2))* , in merge sort there are not nested for loops and the while loops are iterated over half array (size=n/2 and so it cost O(n)). Due to this, the complexity decreases with respect to the other sort algorithms. I claim that merge sort is a "divide et impera" based algorithm, the "divide" part cost *logn* because it halves the array until it reaches size of 1 and the "impera" cost $O(n)$, giving a the end a complexity of order $O(n*logn)$.

#### 7.5 MapReduce implementation

This code was implemented with pySpark via AWS with EMR, so it is not possible to run it on a python file.
Here I have attached the code and the output from pyspark

In [2]:
applicants=sc.textFile('s3://applicantsnicolaspaola/ApplicantsInfo.txt') #from my bucket

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#let's see what we are using
applicants.count()
applicants.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['50000 1000']

In [4]:
first=applicants.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
students,grades=applicants.take(1)[0].split()
grades=int(grades)
grades

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1000

In [14]:
#Same function used above in sort algorithms
def unlist(l):
    for i in l:
        print(*i) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
output=applicants.filter(lambda row: row!=first[0])\
.map(lambda row: row.split())\
.map(lambda row: (sum([float(vote) for vote in row[2:]])/(grades), row[0],row[1]))\
.takeOrdered(50000,lambda row:(-row[0],row[1]))

unlist(output)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24.492 Emily Crispin
24.489 Patricia Witten
24.447 Doreen Richmond
24.445 Bruce Johnson
24.437 David Niederberger
24.436 Keisha Keene
24.436 Steven Boston
24.423 Melody Sanchez
24.419 John Johnson
24.417 Marvin Ramirez
24.414 Joshua Reece
24.411 Luisa Young
24.41 Manuel Sullinger
24.407 Edith Lehtonen
24.406 Ida Mccabe
24.404 Betty Kubiak
24.4 Violet Paulino
24.397 Particia Mirabal
24.396 Cara Baird
24.396 Mattie Salinas
24.395 Kelsey Mcneill
24.395 Richard Parker
24.391 Kathleen Whaley
24.39 Latoya Stemp
24.389 Jeffrey Johnson
24.388 Amy Walker
24.388 Marie Wall
24.386 Harry Lupu
24.385 Josephine Young
24.384 Norma Smith
24.383 Mary Nguyen
24.382 Daryl Singer
24.382 Desiree Paul
24.381 Olive Sacco
24.38 Rose Mcpeak
24.378 Amanda Williams
24.378 Jackson Klopfer
24.376 Jeramy Galicia
24.375 David Kerns
24.375 Juan Gonzalez
24.375 Kyle Carlson
24.375 Ruth Heimsness
24.373 Edward Corella
24.373 Khalilah Hensley
24.373 Robert Rector
24.372 Irvin Johnson
24.372 Wilson Senske
24.368 Rhonda B